In [100]:
import os
from glob import glob
from tqdm import tqdm,trange
import numpy as np
import cv2
from imgaug import augmenters as iaa
import Augmentor
from keras.models import Model,Sequential,load_model
from keras.layers import Dense,Flatten,Reshape,Dropout,MaxPooling2D,Conv2D,BatchNormalization
from keras.utils import to_categorical
from keras import backend as K

import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
datapath = "../Data/fwdsumpics"
train_test_split = 0.75
epochs = 50
classes = 12

In [42]:
def process_new_images():
    filelist = glob("../Data/fwdsumpics/*.jpg")
    for f in tqdm(filelist):
        bn = os.path.basename(f)
        rollno = int(bn[5:8])
        newfolder = datapath+"/{}".format(rollno)
        if not os.path.exists(newfolder):
            os.mkdir(newfolder)
        newpath = newfolder+"/"+bn
        os.rename(f,newpath)
        
def split_new_images():
    ifw = glob("../Data/fwdsumpics/new_img/*")
    for fold in ifw:
        bnf = os.path.basename(fold)
        traindir = "../Data/fwdsumpics/train/{}".format(bnf)
        testdir = "../Data/fwdsumpics/test/{}".format(bnf)
        try:
            os.mkdir(traindir)
            os.mkdir(testdir)
        except:
            pass
        fglob = glob(os.path.join(fold,"*.jpg"))
        spl = int(train_test_split*len(fglob))
        for fil in fglob[:spl]:
            os.rename(fil,os.path.join(traindir,os.path.basename(fil)))
        for fil in fglob[spl:]:
            os.rename(fil,os.path.join(testdir,os.path.basename(fil)))

In [98]:
def contrast_augment():
    folg = glob("../Data/fwdsumpics/train/*")
    for fol in folg:
        cnt = 0
        filg = glob(fol+"/*.jpg")
        fils = []
        for fil in filg:
            img = cv2.imread(fil,cv2.IMREAD_GRAYSCALE)
            fils.append(img)
        for i in range(9):    
            cpipe = iaa.Add(value=np.random.randint(-50,151))
            aug = cpipe.augment_images(fils)
            for j in aug:
                cv2.imwrite(fol+"/{:03d}.jpg".format(cnt),j)
                cnt+=1

In [30]:
model = Sequential()
model.add(Conv2D(32,3,padding='same',activation='relu',input_shape = (64,64,1)))
model.add(BatchNormalization())
model.add(Conv2D(32,3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Conv2D(64,3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Conv2D(128,3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Conv2D(256,3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(classes,activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 64, 64, 32)        320       
_________________________________________________________________
batch_normalization_21 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 64, 64, 32)        9248      
_________________________________________________________________
batch_normalization_22 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_23 (Batc (None, 32, 32, 64)        256       
__________

In [101]:
trainglob = glob("../Data/fwdsumpics/train/**/*.jpg")
labels = [i//80 for i in range(len(trainglob))]
labels = to_categorical(labels)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
p = Augmentor.keras_generator_from_array()